In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import r2_score

%matplotlib inline

In [9]:
train = pd.read_csv('/Users/katychow/DSI/Projects/project-2/datasets/train.csv')
test = pd.read_csv('/Users/katychow/DSI/Projects/project-2/datasets/test.csv')

train = train[(train['Gr Liv Area']<4000) & (train['Lot Area']<25000)]

In [10]:
Nominal = ['Sale Type','Neighborhood','Bldg Type', 'House Style','Foundation'] # no sale condition from jse site
# Bldg Type & Sale Type
# split bldg type - 1Fam, TwnhmE, Else
# split Sale Type - New, Oth&CWD, Else
# I want to use neighborhood, but will need extra time to group based on neighborhood prices (pivots)
Numerical = ['Lot Area','Year Built', 'Year Remod/Add', 
             'BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF',
             '2nd Flr SF','Gr Liv Area', 'Bsmt Full Bath',
             'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
             'Kitchen AbvGr','TotRms AbvGrd',
             'Garage Area','Wood Deck SF', 'Open Porch SF','Screen Porch',
             'Mo Sold', 'Yr Sold' ,'SalePrice']
Ordinal = ['Overall Qual','Overall Cond','Paved Drive']
# only use overall 'overall qual'
Target = ['SalePrice']
Ids = ['Id']

### examples of stuff

In [4]:
'straw' in 'strawberry jam'

True

In [5]:
if 'straw' in ['strawberry', 'jam']:
    print(True)
else:
    print(False)

False


In [11]:
def bin_quality(val):
    if val < 5:
        return 0
    else:
        return 1
    
#train['new+colname'].apply

#train['new_col'] =
train['Overall Qual'].apply(bin_quality)
# train['Overall Qual'].value_counts()

# plt.scatter(train['Overall Qual'].apply(bin_quality),train['SalePrice'],color='pink');

def sale_type_cat(val):
    if val in ['New']:
        return 1
    elif val in ['Oth','CWD']:
        return 2
    else:
        return 0
        
# train['Sale Type'].apply(sale_type_cat)

### make things work

In [16]:
final_train = train[['Id','Lot Area', 'Year Built', 'Year Remod/Add', 'Gr Liv Area',
                  'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
                  'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 
                   'Wood Deck SF','Open Porch SF','Screen Porch','Neighborhood','House Style','Foundation',
                   'Mo Sold', 'Yr Sold' ,'Overall Cond', 'Bldg Type','Sale Type','SalePrice']]

## Dummifying Neighborhood, HouseStyle, Foundation
final_train = pd.get_dummies(final_train,columns=['Neighborhood','House Style','Foundation'],drop_first=True)

In [17]:
final_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,2021.0,1469.889659,843.886190,3.0,749.0,1472.0,2193.0,2930.0
Lot Area,2021.0,9497.982682,3644.495244,1300.0,7488.0,9350.0,11375.0,24572.0
Year Built,2021.0,1971.820881,30.225073,1872.0,1953.0,1975.0,2001.0,2010.0
Year Remod/Add,2021.0,1984.218209,21.019166,1950.0,1965.0,1993.0,2004.0,2010.0
Gr Liv Area,2021.0,1489.630381,481.696185,334.0,1126.0,1440.0,1728.0,3672.0
Bsmt Full Bath,2020.0,0.422772,0.520483,0.0,0.0,0.0,1.0,3.0
Bsmt Half Bath,2020.0,0.063366,0.251680,0.0,0.0,0.0,0.0,2.0
Full Bath,2021.0,1.576447,0.546564,0.0,1.0,2.0,2.0,4.0
Half Bath,2021.0,0.370114,0.501066,0.0,0.0,0.0,1.0,2.0
Bedroom AbvGr,2021.0,2.841168,0.821739,0.0,2.0,3.0,3.0,8.0


In [18]:
final_train['Tot_bath_abv_grd'] = final_train['Full Bath']+0.5*final_train['Half Bath']
final_train['Tot_bath_bsmt']=final_train['Bsmt Full Bath']+0.5*final_train['Bsmt Half Bath']
final_train['Outdoor Liv Area']=final_train['Wood Deck SF']+final_train['Open Porch SF']+final_train['Screen Porch']

final_train['Overall Cond Bi']=final_train['Overall Cond'].map(lambda x: 1 if x >= 5 else 0)
final_train['Bldg Type Bi']=final_train['Bldg Type'].map(lambda x: 1 if x in ['1Fam','TwnhmE'] else 0)
final_train['Sale Type Bi']=final_train['Sale Type'].apply(sale_type_cat)

In [19]:
final_train.head(2)

,Id,Lot Area,Year Built,Year Remod/Add,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,...,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Tot_bath_abv_grd,Tot_bath_bsmt,Outdoor Liv Area,Overall Cond Bi,Bldg Type Bi,Sale Type Bi
0,109,13517,1976,2005,1479,0.0,0.0,2,1,3,...,0,0,0,0,2.5,0.0,44,1,1,0
1,544,11492,1996,1997,2122,1.0,0.0,2,1,4,...,1,0,0,0,2.5,1.0,74,1,1,0


In [20]:
final_train.columns

Index(['Id', 'Lot Area', 'Year Built', 'Year Remod/Add', 'Gr Liv Area',
       'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Wood Deck SF',
       'Open Porch SF', 'Screen Porch', 'Mo Sold', 'Yr Sold', 'Overall Cond',
       'Bldg Type', 'Sale Type', 'SalePrice', 'Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr',
       'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_GrnHill',
       'Neighborhood_IDOTRR', 'Neighborhood_Landmrk', 'Neighborhood_MeadowV',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer',
       'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr',
       'Neighb

In [21]:
final_features = ['Id', 'Lot Area', 'Year Built', 'Year Remod/Add', 'Gr Liv Area',
             'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Mo Sold', 'Yr Sold',
             'Outdoor Liv Area',  'Tot_bath_abv_grd', 'Tot_bath_bsmt','Overall Cond Bi',
             'Bldg Type Bi','Sale Type Bi','Neighborhood_Blueste',
               'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr',
               'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
               'Neighborhood_Gilbert', 'Neighborhood_Greens', 'Neighborhood_GrnHill',
               'Neighborhood_IDOTRR', 'Neighborhood_Landmrk', 'Neighborhood_MeadowV',
               'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
               'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
               'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer',
               'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr',
               'Neighborhood_Timber', 'Neighborhood_Veenker', 'House Style_1.5Unf',
               'House Style_1Story', 'House Style_2.5Fin', 'House Style_2.5Unf',
               'House Style_2Story', 'House Style_SFoyer', 'House Style_SLvl',
               'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab',
               'Foundation_Stone', 'Foundation_Wood','SalePrice']
final_feats_train = final_train[final_features]

In [22]:
final_feats_train.dropna(inplace = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
final_feats_train.shape

(2020, 56)

### Try following steps 
 * test/train split 
 * polynomial features
 * standard scaler
 * fit Lasso/Ridge

In [42]:
mod_feats = ['Lot Area', 'Year Built', 'Year Remod/Add', 'Gr Liv Area',
             'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Mo Sold', 'Yr Sold',
             'Outdoor Liv Area',  'Tot_bath_abv_grd', 'Tot_bath_bsmt','Overall Cond Bi',
             'Bldg Type Bi','Neighborhood_Blueste',
       'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr',
       'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_Edwards',
       'Neighborhood_Gilbert', 'Neighborhood_Greens', # 'Neighborhood_GrnHill',
       'Neighborhood_IDOTRR',  'Neighborhood_MeadowV', #'Neighborhood_Landmrk',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NPkVill',
       'Neighborhood_NWAmes', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
       'Neighborhood_OldTown', 'Neighborhood_SWISU', 'Neighborhood_Sawyer',
       'Neighborhood_SawyerW', 'Neighborhood_Somerst', 'Neighborhood_StoneBr',
       'Neighborhood_Timber', 'Neighborhood_Veenker', 'House Style_1.5Unf',
       'House Style_1Story', 'House Style_2.5Fin', 'House Style_2.5Unf',
       'House Style_2Story', 'House Style_SFoyer', 'House Style_SLvl',
       'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab',
       'Foundation_Stone', 'Foundation_Wood','Sale Type Bi']
X = final_feats_train[mod_feats]
y = final_feats_train['SalePrice']

In [43]:
pf = PolynomialFeatures(degree = 2, include_bias=False)
pf = pf.fit(X)
X_poly = pf.transform(X)

In [44]:
X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(X_poly,y,test_size = 0.2, random_state=42)

In [45]:
ss = StandardScaler()
ss.fit(X_poly_train)

X_poly_train_ss = ss.transform(X_poly_train)
X_poly_test_ss = ss.transform(X_poly_test)

In [46]:
y_log_train = np.log(y_poly_train)

In [47]:
lasso = LassoCV(max_iter=10000)
ridge = RidgeCV()

In [48]:
print(cross_val_score(lasso, X_poly_train_ss,y_log_train, cv = 5))
print(cross_val_score(lasso, X_poly_train_ss,y_log_train,cv = 5).mean())

# print(cross_val_score(ridge, X_poly_train_ss,y_log_train, cv = 5))
# print(cross_val_score(ridge, X_poly_train_ss,y_log_train,cv = 5).mean())

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[0.81294907 0.80568767 0.87928635 0.85216281 0.8879135 ]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


0.8475998800815467


In [49]:
lasso.fit(X_poly_train_ss,y_log_train)

print(lasso.score(X_poly_train_ss,y_log_train))

lasso_pred = lasso.predict(X_poly_test_ss)
lasso_pred = np.exp(lasso_pred)
r2_score(y_poly_test,lasso_pred)

0.8828311264074535


0.8880073512408004

In [50]:
ridge.fit(X_poly_train_ss,y_log_train)

print(ridge.score(X_poly_train_ss,y_log_train))

ridge_pred = ridge.predict(X_poly_test_ss)
ridge_pred = np.exp(ridge_pred)
r2_score(y_poly_test,ridge_pred)

0.9326984508910637


0.8764844828968161

Somehow this is really taxing on the computer so will not try for this iteration

```python
enet_alphas = np.arange(0.01, 5.0, 0.005)

# Set up our l1 ratio. (What does this do?) 
# enet_ratio = np.arange(0.01, 1.0, 0.01)  
# this will weight lasso more heavily as ratio -> 1

# Instantiate model.
enet_model = ElasticNetCV(l1_ratio=enet_ratio, cv=5, max_iter=10000)

# Fit model using optimal alpha.
enet_model = enet_model.fit(X_poly_train_ss, y_log_train)

print(enet_model.score(X_poly_train_ss,y_log_train))

enet_pred = enet_model.predict(X_poly_test_ss)
enet_pred = np.exp(enet_pred)
r2_score(y_poly_test,enet_pred)
```

### now to fit on testing data and create predictions
* create all transformations
* create all polynomials
* scale 
* predict
* save predictions 
* submit

In [51]:
final_test = test[['Id','Lot Area', 'Year Built', 'Year Remod/Add', 'Gr Liv Area',
                  'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
                  'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 
                   'Wood Deck SF','Open Porch SF','Screen Porch','Neighborhood','House Style','Foundation',
                   'Mo Sold', 'Yr Sold' ,'Overall Cond', 'Bldg Type','Sale Type']]

final_test = pd.get_dummies(final_test,columns=['Neighborhood','House Style','Foundation'],drop_first=True)

In [52]:
final_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,879.0,1445.588168,850.717105,2.0,692.5,1435.0,2197.0,2928.0
Lot Area,879.0,10340.920364,10047.335167,1477.0,7298.5,9453.0,11606.5,215245.0
Year Built,879.0,1970.533561,30.403527,1880.0,1954.0,1972.0,2000.0,2010.0
Year Remod/Add,879.0,1984.444824,20.454546,1950.0,1967.0,1992.0,2003.0,2010.0
Gr Liv Area,879.0,1500.531286,517.413260,407.0,1114.0,1436.0,1781.0,4676.0
Bsmt Full Bath,879.0,0.440273,0.529983,0.0,0.0,0.0,1.0,2.0
Bsmt Half Bath,879.0,0.055745,0.229560,0.0,0.0,0.0,0.0,1.0
Full Bath,879.0,1.541524,0.560908,0.0,1.0,2.0,2.0,4.0
Half Bath,879.0,0.399317,0.506045,0.0,0.0,0.0,1.0,2.0
Bedroom AbvGr,879.0,2.879408,0.830249,0.0,2.0,3.0,3.0,6.0


In [53]:
final_test['Tot_bath_abv_grd'] = final_test['Full Bath']+0.5*final_test['Half Bath']
final_test['Tot_bath_bsmt']=final_test['Bsmt Full Bath']+0.5*final_test['Bsmt Half Bath']
final_test['Outdoor Liv Area']=final_test['Wood Deck SF']+final_test['Open Porch SF']+final_test['Screen Porch']

final_test['Overall Cond Bi']=final_test['Overall Cond'].map(lambda x: 1 if x >= 5 else 0)
final_test['Bldg Type Bi']=final_test['Bldg Type'].map(lambda x: 1 if x in ['1Fam','TwnhmE'] else 0)
final_test['Sale Type Bi']=final_test['Sale Type'].apply(sale_type_cat)

In [54]:
final_test_mod = final_test[mod_feats]

In [55]:
final_test_ss = ss.transform(pf.transform(final_test_mod))

In [56]:
final_pred = lasso.predict(final_test_ss)
final_pred = np.exp(final_pred)

In [57]:
pd.DataFrame({
    'Id': final_test['Id'],
    'SalePrice': final_pred
})

,Id,SalePrice
0,2658,118742.346275
1,2718,129672.641709
2,2414,207376.811061
3,1989,104448.746342
4,625,163950.720926
5,333,92258.714662
6,1327,122175.707617
7,858,154649.519971
8,95,184609.272638
9,1568,162551.182968


In [59]:
pd.DataFrame({
    'Id': final_test['Id'],
    'SalePrice': final_pred
}).to_csv('/Users/katychow/DSI/Projects/project-2/Submissions/katy_chow_submission3_housingproject2_20181204.csv', index = False)